# RFM Analysis

This iPython notebook explains how to perform RFM analysis from customer purchase history data. The sample orders file is Sample - Superstore dataset from Tableau Software.

In [21]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

Read the sample orders file, containing all past purchases for all customers.

In [22]:
orders = pd.read_csv('RFM.csv',sep=',')

In [6]:
orders.head()

,id,LastLog_days_ago,Payments,Money
0,1,19,1,1.00
1,2,2,25,27.77
2,3,18,1,4.99
3,4,19,2,2.98
4,5,2,4,61.97


In [7]:
orders.describe()

,id,LastLog_days_ago,Payments,Money
count,6344.000000,6344.000000,6344.000000,6344.000000
mean,3172.500000,8.228405,9.423707,115.328695
std,1831.499386,8.053561,19.127101,537.277480
min,1.000000,1.000000,1.000000,0.990000
25%,1586.750000,2.000000,2.000000,3.980000
50%,3172.500000,4.000000,4.000000,14.980000
75%,4758.250000,12.000000,9.000000,60.872500
max,6344.000000,30.000000,435.000000,24194.420000


## Create the RFM Table

Create the RFM Table

In [9]:
rfmTable = orders

## Validating the RFM Table

In [10]:
rfmTable.head()

,id,LastLog_days_ago,Payments,Money
0,1,19,1,1.00
1,2,2,25,27.77
2,3,18,1,4.99
3,4,19,2,2.98
4,5,2,4,61.97


## Determining RFM Quartiles

In [11]:
quantiles = rfmTable.quantile(q=[0.25,0.5,0.75])

In [12]:
quantiles

,id,LastLog_days_ago,Payments,Money
0.25,1586.75,2.0,2.0,3.9800
0.50,3172.50,4.0,4.0,14.9800
0.75,4758.25,12.0,9.0,60.8725


In [13]:
quantiles = quantiles.to_dict()

In [14]:
quantiles

{'LastLog_days_ago': {0.25: 2.0, 0.5: 4.0, 0.75: 12.0},
 'Money': {0.25: 3.98, 0.5: 14.98, 0.75: 60.872499999999995},
 'Payments': {0.25: 2.0, 0.5: 4.0, 0.75: 9.0},
 'id': {0.25: 1586.75, 0.5: 3172.5, 0.75: 4758.25}}

## Creating the RFM segmentation table

In [15]:
rfmSegmentation = rfmTable

We create two classes for the RFM segmentation since, being high recency is bad, while high frequency and monetary value is good. 

In [16]:
# Arguments (x = value, p = recency, monetary_value, frequency, k = quartiles dict)
def RClass(x,p,d):
    if x <= d[p][0.25]:
        return 1
    elif x <= d[p][0.50]:
        return 2
    elif x <= d[p][0.75]: 
        return 3
    else:
        return 4
    
# Arguments (x = value, p = recency, monetary_value, frequency, k = quartiles dict)
def FMClass(x,p,d):
    if x <= d[p][0.25]:
        return 4
    elif x <= d[p][0.50]:
        return 3
    elif x <= d[p][0.75]: 
        return 2
    else:
        return 1


In [18]:
rfmSegmentation['R_Quartile'] = rfmSegmentation['LastLog_days_ago'].apply(RClass, args=('LastLog_days_ago',quantiles,))
rfmSegmentation['F_Quartile'] = rfmSegmentation['Payments'].apply(FMClass, args=('Payments',quantiles,))
rfmSegmentation['M_Quartile'] = rfmSegmentation['Money'].apply(FMClass, args=('Money',quantiles,))

In [19]:
rfmSegmentation['RFMClass'] = rfmSegmentation.R_Quartile.map(str) \
                            + rfmSegmentation.F_Quartile.map(str) \
                            + rfmSegmentation.M_Quartile.map(str)

In [25]:
rfmSegmentation.head()

,id,LastLog_days_ago,Payments,Money,R_Quartile,F_Quartile,M_Quartile,RFMClass
0,1,19,1,1.00,4,4,4,444
1,2,2,25,27.77,1,1,2,112
2,3,18,1,4.99,4,4,3,443
3,4,19,2,2.98,4,4,4,444
4,5,2,4,61.97,1,3,1,131


In [19]:
# Uncomment any of the following lines to: copy data to clipboard or save it to a CSV file.
# rfmSegmentation.to_clipboard()
# rfmSegmentation.to_csv('rfm-table.csv', sep=',')

Who are the top 5 best customers? by RFM Class (111), high spenders who buy recently and frequently?

In [29]:
rfmSegmentation[rfmSegmentation['RFMClass']=='111'].sort_values(by='Money', ascending=False).head(5)

,id,LastLog_days_ago,Payments,Money,R_Quartile,F_Quartile,M_Quartile,RFMClass
1325,1326,1,207,24194.42,1,1,1,111
1907,1908,2,177,16512.33,1,1,1,111
1515,1516,2,323,6384.77,1,1,1,111
4798,4799,2,93,4745.08,1,1,1,111
273,274,2,64,4692.40,1,1,1,111
